In [ ]:
!pip install --upgrade duckdb pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c7d9b83354269c82bc04fd7d042d81917ccbf41348f26f67e1b19c63dfd958e9
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# We should have the same version (0.10.1) of duckdb to load the database without any problem:
!pip show duckdb

Name: duckdb
Version: 0.10.1
Summary: DuckDB in-process database
Home-page: https://www.duckdb.org
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: malloy


In [ ]:
!wget -O "duckdb.jar" "https://repo1.maven.org/maven2/org/duckdb/duckdb_jdbc/0.10.1/duckdb_jdbc-0.10.1.jar"

--2024-04-16 15:46:22--  https://repo1.maven.org/maven2/org/duckdb/duckdb_jdbc/0.10.1/duckdb_jdbc-0.10.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64009472 (61M) [application/java-archive]
Saving to: ‘duckdb.jar’

duckdb.jar          100%[===================>]  61.04M   192MB/s    in 0.3s    

2024-04-16 15:46:23 (192 MB/s) - ‘duckdb.jar’ saved [64009472/64009472]



In [ ]:
import duckdb

# Conectar a la base de datos DuckDB
conn = duckdb.connect(database='database_new.duckdb', read_only=True)

# Ejecutar una consulta para obtener la estructura de la tabla 'flights'
query = "PRAGMA table_info('flights')"
table_info = conn.execute(query).fetchall()

# Imprimir información de cada columna
for column in table_info:
    print(f"Column Name: {column[1]}, Data Type: {column[2]}")

# Cerrar la conexión
conn.close()


Column Name: airport_acronym, Data Type: VARCHAR
Column Name: longitude, Data Type: DOUBLE
Column Name: latitude, Data Type: DOUBLE
Column Name: city, Data Type: VARCHAR
Column Name: flight_number_default, Data Type: VARCHAR
Column Name: estimated_arrival_time, Data Type: VARCHAR
Column Name: estimated_departure_time, Data Type: VARCHAR
Column Name: real_arrival_time, Data Type: VARCHAR
Column Name: real_departure_time, Data Type: VARCHAR
Column Name: scheduled_arrival_time, Data Type: VARCHAR
Column Name: scheduled_departure_time, Data Type: VARCHAR
Column Name: owner_name, Data Type: VARCHAR
Column Name: owner_logo, Data Type: VARCHAR
Column Name: destination_gate, Data Type: VARCHAR
Column Name: destination_terminal, Data Type: VARCHAR
Column Name: destination_baggage, Data Type: VARCHAR
Column Name: origin_name, Data Type: VARCHAR
Column Name: origin_city, Data Type: VARCHAR
Column Name: origin_latitude, Data Type: VARCHAR
Column Name: origin_longitude, Data Type: VARCHAR
Column Na

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

DF = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:database_new.duckdb") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT * FROM flights") \
  .load()

DF.show()

+---------------+---------+---------+---------+---------------------+----------------------+------------------------+-----------------+-------------------+----------------------+------------------------+--------------------+--------------------+----------------+--------------------+-------------------+--------------------+-----------+---------------+----------------+-----------+---------------+------------------+
|airport_acronym|longitude| latitude|     city|flight_number_default|estimated_arrival_time|estimated_departure_time|real_arrival_time|real_departure_time|scheduled_arrival_time|scheduled_departure_time|          owner_name|          owner_logo|destination_gate|destination_terminal|destination_baggage|         origin_name|origin_city|origin_latitude|origin_longitude|origin_gate|origin_terminal|flight_status_live|
+---------------+---------+---------+---------+---------------------+----------------------+------------------------+-----------------+-------------------+-----------

## Preprocessing

In [ ]:
from pyspark.sql import functions as F

#### Missings

In [ ]:
null_counts = DF.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in DF.columns])
null_counts.show()

+---------------+---------+--------+----+---------------------+----------------------+------------------------+-----------------+-------------------+----------------------+------------------------+----------+----------+----------------+--------------------+-------------------+-----------+-----------+---------------+----------------+-----------+---------------+------------------+
|airport_acronym|longitude|latitude|city|flight_number_default|estimated_arrival_time|estimated_departure_time|real_arrival_time|real_departure_time|scheduled_arrival_time|scheduled_departure_time|owner_name|owner_logo|destination_gate|destination_terminal|destination_baggage|origin_name|origin_city|origin_latitude|origin_longitude|origin_gate|origin_terminal|flight_status_live|
+---------------+---------+--------+----+---------------------+----------------------+------------------------+-----------------+-------------------+----------------------+------------------------+----------+----------+----------------+

In [ ]:
# We decide to delete those rows with missings in arrivals variable because it doesn't give any information to us
DF = DF.dropna(subset=["arrivals"])

In [ ]:
null_counts_post_cleanup = DF.select([F.count(F.when(F.col("arrivals").isNull(), "arrivals")).alias("null_arrivals")])
null_counts_post_cleanup.show()

+-------------+
|null_arrivals|
+-------------+
|            0|
+-------------+

